# Initialization

In [2]:
import os
import cv2
import fnv
import fnv.file
import fnv.reduce
import numpy as np
from pprint import PrettyPrinter
pprint = PrettyPrinter().pprint

video_path = "C:\\Users\\U361220\\Desktop\\SEQ Conversion\\Videos\\TermovisionDamage1.seq"
im_counts = fnv.file.ImagerFile(video_path)  # reader for COUNTS unit
im_temp = fnv.file.ImagerFile(video_path)  # reader for TEMPERATURE_FACTORY unit
im_signal = fnv.file.ImagerFile(video_path)  # reader for OBJECT_SIGNAL unit
# Why do we need three image readers, I hear you ask?
# Because if you use the same one for all three units,
# when you come to change the unit for the second time,
# then whatever you change the the third unit to, the third 
# im.final will be the same as the second im.final. Except
# if you use COUNTS. Very weird.

print(list(im_counts.supported_units))

[<Unit.COUNTS: 0>, <Unit.RADIANCE_FACTORY: 4>, <Unit.OBJECT_SIGNAL: 3>, <Unit.TEMPERATURE_FACTORY: 5>]


In [2]:
concated_frames = np.empty((im_counts.height, im_counts.width, 3))
concated_frames.shape

(480, 640, 3)

# Concatenation with regular frames

In [3]:
output_folder = "C:\\Users\\U361220\\Desktop\\SEQ Conversion\\Output\\SDK\\Damage1\\Normal Frames\\Counts_Temp_Signal\\UINT16"
scale = np.uint16

In [23]:
# Iterate over all frames in the video
for i in range(im_counts.num_frames):
    im_counts.unit = fnv.Unit.COUNTS  # Set the desired unit
    im_counts.get_frame(i)  # Initialize im_counts.final
    im_min = min(im_counts.final)
    im_max = max(im_counts.final)
    im_range = im_max - im_min
    counts_final_unscaled = np.array(im_counts.final, copy=False).reshape((im_counts.height, im_counts.width))  # Reshape frame values
    counts_final = (((counts_final_unscaled - im_min) / im_range) * 255).astype(scale)  # Scale for jpeg
    
    # Repeat for temperature unit
    im_temp.unit = fnv.Unit.TEMPERATURE_FACTORY
    im_temp.get_frame(i)
    im_min = min(im_temp.final)
    im_max = max(im_temp.final)
    im_range = im_max - im_min
    temp_final_unscaled = np.array(im_temp.final, copy=False).reshape((im_temp.height, im_temp.width))
    temp_final = (((temp_final_unscaled - im_min) / im_range) * 255).astype(scale)

    # Repeat for radiance unit
    im_signal.unit = fnv.Unit.OBJECT_SIGNAL
    im_signal.get_frame(i)
    im_min = min(im_signal.final)
    im_max = max(im_signal.final)
    im_range = im_max - im_min
    signal_final_unscaled = np.array(im_signal.final, copy=False).reshape((im_signal.height, im_signal.width))
    signal_final = (((signal_final_unscaled - im_min) / im_range) * 255).astype(scale)

    # Reshape into a (480, 640, 3) array for jpg
    concated_frames = np.array([counts_final, temp_final, signal_final]).swapaxes(0, 2).swapaxes(1, 0)
    
    # Save the image
    cv2.imwrite(f'{output_folder}\\concat_frame_{i}.jpeg', concated_frames)

# Concatenation with superframes

In [26]:
output_folder = "C:\\Users\\U361220\\Desktop\\SEQ Conversion\\Output\\SDK\\Damage1\\Superframes\\Counts_Temp_Signal\\UINT8"
scale = np.uint8

In [27]:
# Get a superframe iterator
superframe_iter = im_counts.frame_iter(fnv.Preset.SUPERFRAME)

# Go over each superframe
for frame in superframe_iter:
    i = frame.frame_info.frame
    
    im_counts.unit = fnv.Unit.COUNTS  # Set the desired unit
    im_counts.get_superframe(i)  # Initialize im_counts.final
    im_min = min(im_counts.final)
    im_max = max(im_counts.final)
    im_range = im_max - im_min
    counts_final_unscaled = np.array(im_counts.final, copy=False).reshape((im_counts.height, im_counts.width))  # Reshape frame values
    counts_final = (((counts_final_unscaled - im_min) / im_range) * 255).astype(scale)  # Scale for jpeg
    
    # Repeat for temperature unit
    im_temp.unit = fnv.Unit.TEMPERATURE_FACTORY
    im_temp.get_superframe(i)
    im_min = min(im_temp.final)
    im_max = max(im_temp.final)
    im_range = im_max - im_min
    temp_final_unscaled = np.array(im_temp.final, copy=False).reshape((im_temp.height, im_temp.width))
    temp_final = (((temp_final_unscaled - im_min) / im_range) * 255).astype(scale)

    # Repeat for radiance unit
    im_signal.unit = fnv.Unit.OBJECT_SIGNAL
    im_signal.get_superframe(i)
    im_min = min(im_signal.final)
    im_max = max(im_signal.final)
    im_range = im_max - im_min
    signal_final_unscaled = np.array(im_signal.final, copy=False).reshape((im_signal.height, im_signal.width))
    signal_final = (((signal_final_unscaled - im_min) / im_range) * 255).astype(scale)

    # Reshape into a (480, 640, 3) array for jpg
    concated_frames = np.array([counts_final, temp_final, signal_final]).swapaxes(0, 2).swapaxes(1, 0)
    
    # Save the image
    cv2.imwrite(f'{output_folder}\\concat_frame_{i}.jpeg', concated_frames)


# Adding a Colormap

In [13]:
image_folder = "C:\\Users\\U361220\\Desktop\\SEQ Conversion\\Output\\SDK\\Damage1\\Superframes\\Counts_Temp_Signal\\UINT16"
output_folder = "C:\\Users\\U361220\\Desktop\\SEQ Conversion\\Output\\Color Mapping\\Damage1\\Superframes\\UINT16"
colormap = cv2.COLORMAP_INFERNO  # Clostst thing to thermal studio

In [14]:
def get_images():
    image_list = []
    # Read in images from IMAGE_DIR
    for filename in os.listdir(image_folder):
        if filename.endswith('jpeg'):
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                image_list.append(image)
    # Return them as a list
    return image_list

images = get_images()

for i, image in enumerate(images):
    # Make the image coloured
    color_image = cv2.applyColorMap(image, colormap)
    image_path = os.path.join(output_folder, f"colormapped_image_{i}.jpg")
    0
    # Save the image
    cv2.imwrite(image_path, color_image)

# Adding a temperature scale bar

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load your image with OpenCV. It's assumed that the image is already in temperature units
img = cv2.imread('your_image_path.jpg', cv2.IMREAD_GRAYSCALE)

# Set min and max temperature values
min_temp = -30  # change to your actual min temperature value
max_temp = 50  # change to your actual max temperature value

# Map the temperature values to the range [0, 255]
img_mapped = ((img - min_temp) / (max_temp - min_temp) * 255).astype(np.uint8)

# Apply a colormap
colored_img = cv2.applyColorMap(img_mapped, cv2.COLORMAP_JET)

# Now we create a colorbar using matplotlib
plt.figure(figsize=(0.5, 4))

# We use imshow to show the colorbar alone, it uses the same colormap as the image
img = plt.imshow(np.linspace(0, 255, 256)[None, :], aspect='auto', cmap=plt.get_cmap('jet'), origin='lower')

# Set the colorbar limits
img.set_clim(vmin=min_temp, vmax=max_temp)

plt.colorbar(img, orientation='vertical')

plt.savefig('colorbar.jpg', bbox_inches='tight', pad_inches=0)

# Load the colorbar image with OpenCV
colorbar = cv2.imread('colorbar.jpg')

# Resize the colorbar to desired height. Here I'm making it the same height as the input image
colorbar = cv2.resize(colorbar, (colorbar.shape[1], colored_img.shape[0]))

# Concatenate the colorbar horizontally with the image
final_image = cv2.hconcat([colored_img, colorbar])

# Show the final image
cv2.imshow('Image with temperature scale', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
